In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%load_ext memory_profiler

In [ ]:
import pandas as pd
from utils.import_utils import *
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from src.feature_processing import *
from src.item_processing import  *
from src.utils.stats_utils import *
import plotly.express as px
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
with initialize(config_path='../configuration', version_base='1.1'):
    config = compose(config_name='main.yaml')

In [ ]:
features_class = ItemFeatureProcessing(config)

In [ ]:
df_item = features_class.df_item
self = features_class

In [ ]:
filter_condition = (df_item['type']=='MultyOptionsQuestion')
data = df_item[filter_condition]
pivot_table = pd.DataFrame( data.responsible.unique(), columns = ['responsible'])
for variable_name in data.variable_name.unique():
    df_values = pd.get_dummies(data[data['variable_name'] == variable_name]['value'].explode()).groupby(level=0).sum()
    # Joining back the exploded values to the original dataframe
    df = data[data['variable_name'] == variable_name][['responsible','value']].drop('value', axis=1).join(df_values)

    # Function to calculate entropy
    def calculate_entropy(row):
        values = row.values
        values = values[values > 0]  # Filter out zero values
        probabilities = values / values.sum()
        entropy = -np.sum(probabilities * np.log2(probabilities)) * len(values)
        return entropy

    # # Calculating entropy grouped by 'variable_name' and 'responsible'
    result = df.groupby(['responsible']).apply(calculate_entropy).reset_index()
    result.columns = ['responsible', 'entropy']
    pivot_table[variable_name] = pivot_table['responsible'].map(result.set_index('responsible')['entropy'])
    

In [ ]:
entropy_columns = [col for col in pivot_table.columns if 'entropy' in col]
pivot_table['entropy__multipleoption_question'] = pivot_table.drop(columns=['responsible']).mean(1)
entropy_ = pivot_table.groupby('responsible')['entropy__multipleoption_question'].mean()
entropy_.plot(kind='bar')
plt.title('entropy__multipleoption_question')
plt.show()